In [2]:
import pyautogui
import cv2
import math
import numpy as np # numpy 패키지 로드하여 np로 사용
import torch
from PIL import ImageGrab
import pytesseract

In [3]:
#setup
webcamYN = False
pyautoguiYn = True

textYN = True
PytesseractYN = True

In [4]:
# 모델 로드 (pre-trained 모델 사용) 사람 차량등 사물 인식
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x' 가능


Using cache found in C:\Users\siwon/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-9-3 Python-3.11.9 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
# 캡쳐 시작 좌표, 가로, 세로 길이 4k 모니터에서 좌측 유튜브 영상 기준
x, y, width, height = 10, 195, 1890, 1000

#윈도우 화면 캡쳐
def monitorCapture(x, y, width, height):
    #im = pyautogui.screenshot(region=(x, y, width, height))
    im = ImageGrab.grab(bbox=(x, y, x + width, y + height))
    return cv2.cvtColor(np.array(im), cv2.COLOR_BGR2RGB)



In [6]:
#웹캠 설정
global webcamItem
def setupToWebcam():
    webcamItem = cv2.VideoCapture(0)

    # 웹캠이 열리지 않을 경우
    if not webcamItem.isOpened():
        print("Error: Could not open webcam.")
        exit()

def webcam():
    ret, frame = webcamItem.read()

    if not ret:
        print("Error: Could not read frame.")

    # 프레임 크기를 YOLO 모델에 맞게 조정
    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 모델은 RGB 형식의 이미지를 사용합니다.

def webcalRelease():
  webcamItem.release()
if webcamYN:
    setupToWebcam()

In [7]:
def checkoToPytesseract(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  data = pytesseract.image_to_data(gray, output_type=Output.DICT)

  # 추출된 텍스트 박스 그리기
  n_boxes = len(data['text'])
  for i in range(n_boxes):
    if int(data['conf'][i]) > 60:  # 신뢰도 기준으로 필터링
      (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
      cv2.putText(image, data['text'][i], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

In [8]:
while True:
  #캐니 엣지 값
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
  
  img = monitorCapture(x, y, width, height)
  #img = webcam()
   
  # YOLO 모델을 사용해 객체 탐지
  results = model(img)

  #결과 이미지를 얻고, 원본 BGR 형식으로 다시 변환
  img = results.render()[0]  # render()는 탐지된 객체에 박스를 그린 이미지를 반환합니다.
  #if PytesseractYN:
  #  checkoToPytesseract(img)
  # 화면에 결과 표시
  cv2.imshow('YOLOv5 Real-time Detection', img)


In [9]:
# 리소스 해제
if webcamYN:
  webcalRelease
cv2.destroyAllWindows()